# Final (Exam) Project

In this final (exam) project, you will do copy propagation analysis and optimization in uCIR. First, you will need the Fifth Project implemented with basic blocks as start point. You do not need the previous analsysis and optimizations done, although it is preferable to complete this task.

## Data-flow analysis: copy propagation

Supose statements of the form

```
s:  x = y
u:  z = 3 + x
```

Copy propagation would yield:

```
u: z = 3 + y
```

## Copy propagation conditions.

Assume that for a statement <tt>u</tt> where <tt>x</tt> is used

*   statement <tt>s</tt> is the only definition of <tt>y</tt> reaching <tt>u</tt>
*   on every path from <tt>s</tt> to <tt>u</tt> there are no assignments to <tt>y</tt>

Then we may substitute <tt>x</tt> for <tt>y</tt> in <tt>u</tt>.

Observe that

*   we know how to check the first condition,
*   but not the second one ...

We shall set up a new data-flow analysis together with new data-flow equations to solve this problem.

##  Copy reaching a point.

Let ```s: x = y``` be a copy statement and $p$ be a point of the uC program P. We say that the copy ```s: x = y``` reaches $u$ if every path $p$ from the initial node of P to $u$

*   contains ```s: x = y```
*   and subsequent to the last occurrence of ```s: x = y``` on path $u$ there is no assignment to <tt>x</tt> or <tt>y</tt>.

This implies that the statement ```s: x = y``` reaches the point $u$.

##  Copy propagation data-flow sets.

For a basic block B:

*   _in_<sub>C</sub>(B) is the set of the copies (from anywhere in $P$ reaching entry(B).
*   _out_<sub>C</sub>(B) is the set of the copies (from anywhere in $P$ reaching exit(B).
*   _gen_<sub>C</sub>(B) is the set of the copies defined in B and reaching exit(B).
*   A copy statement <tt>s: x = y</tt> is killed in B if
    *   <tt>s: x = y</tt> $ \not\in$ B and
    *   <tt>x</tt> or <tt>y</tt> are assigned in B.
*   _kill_<sub>C</sub>(B) is the set of the copy statements (from anywhere in $P$) that are killed in within block B by a redefinition of <tt>y</tt> or <tt>x</tt>.

Observe that

*   _in_<sub>C</sub>(B) can contain **only one** copy statement with a given <tt>x</tt> on the left.

Observe that for every basic block B:

*   the data-flow sets _gen_<sub>C</sub>(B) and _kill_<sub>C</sub>(B) can be computed easily,
*   the definitions of _gen_<sub>C</sub>(B) and _kill_<sub>C</sub>(B) essentially deals with graph theory and will not detect copy statements that are never hit (= evaluated),
*   so we may apply the following precautionary principles
    *   we add a copy statement to _gen_<sub>C</sub>(B) only if we are sure that it is hit,
    *   we add a copy statement to _kill_<sub>C</sub>(B) even if it is doubtful that it will be killed.


## Data-flow equations

For every basic block B the data-flow sets _in_<sub>C</sub>(B) and _out_<sub>C</sub>(B) satisfy the following equations

*  _in_<sub>C</sub>(B) = $\bigcap_{(B' \in\ pred(B))}$ _out_<sub>C</sub>(B'),


*  _out_<sub>C</sub>(B) = _gen_<sub>C</sub>(B) $\bigcup$ (_in_<sub>C</sub>(B) $\setminus$ _kill_<sub>C</sub>(B))


## Computing the data-flow sets for the copy propagation problem

Assume that the set C of all copy statements in $P$ has been computed. Then, for each basic block B one can compute _gen_<sub>C</sub>(B) and _kill_<sub>C</sub>(B).

Now set

*  $in_{C}(B_{1}) = \phi$ and,

*  $out_{C}(B_{1}) = gen_{C}(B_{1})$


where _B_<sub>1</sub> is the first block.

Then one can set up a completion algorithm initializing every basic block B $\neq$ _B_<sub>1</sub>

*  _in_<sub>C</sub>(B) = C

and using the updating equations

*  _out_<sub>C</sub>(B) = _gen_<sub>C</sub>(B) $\bigcup$ (_in_<sub>C</sub>(B) $\setminus$ _kill_<sub>C</sub>(B))

*  _in_<sub>C</sub>(B) = $\bigcap_{(B' \in\ pred(B))}$ _out_<sub>C</sub>(B')


# Copy Propagation example
```
int main() {
 int x, y, z;
 read(y);
 x = y;
 z = 3 + x;
 print(z);
 return 0;
}
```
uCIR:
```
define_int @main 
entry:
  alloc_int %1 
  alloc_int %x 
  alloc_int %y 
  alloc_int %z 
  read_int %y 
  load_int %y %2 
  store_int %2 %x 
  literal_int 3 %3 
  load_int %x %4 
  add_int %3 %4 %5 
  store_int %5 %z 
  load_int %z %6 
  print_int %6 
  literal_int 0 %7 
  store_int %7 %1 
  jump label %exit
exit:
  load_int %1 %8 
  return_int %8
```
uCIR with copy propagation:
```
define_int @main 
entry:
  alloc_int %1 
  alloc_int %y 
  alloc_int %z 
  read_int %y 
  literal_int 3 %3 
  load_int %y %4 
  add_int %3 %4 %5 
  store_int %5 %z 
  load_int %z %6 
  print_int %6 
  literal_int 0 %7 
  store_int %7 %1 
  jump label %exit
exit:
  load_int %1 %8 
  return_int %8
```